In [ ]:
# UNCOMMENT FOR INTERACTIVE PLOTTING
# %matplotlib notebook
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation, rc, cm
import IPython, io, urllib
import requests
import warnings
import curses
import ipywidgets as widgets
from matplotlib import animation, rc, cm
import time
from snakelib import FastSnake

warnings.filterwarnings("ignore")

rc("animation", html="html5")

# Reinforcement learning on snake with a genetic neural network

In [ ]:
left_widget = widgets.Button(
    description="",
    disabled=False,
    button_style="success",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Want to go left ?",
    icon="fa-arrow-left",
)

right_widget = widgets.Button(
    description="",
    disabled=False,
    button_style="success",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Want to go right ?",
    icon="fa-arrow-right",
)

up_widget = widgets.Button(
    description="",
    disabled=False,
    button_style="success",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Want to go up ?",
    icon="fa-arrow-up",
)


down_widget = widgets.Button(
    description="",
    disabled=False,
    button_style="success",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Want to go down ?",
    icon="fa-arrow-down",
)

reset_widget = widgets.Button(
    description="",
    disabled=False,
    button_style="danger",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Want to reset ?",
    icon="fa-power-off",
)

direction = 0


def set_direction(direction):
    snake.play(direction)
    print(f"played {direction}")
    update_fig()


# def clicked_left(arg):
#     set_direction(2)


def reset_game():
    snake.reset()
    update_fig()


left_widget.on_click(lambda arg: set_direction(2))
right_widget.on_click(lambda arg: set_direction(0))
up_widget.on_click(lambda arg: set_direction(1))
down_widget.on_click(lambda arg: set_direction(3))
reset_widget.on_click(lambda arg: reset_game())


def update_fig():
    im.set_array(snake.grid)
    status = snake.status
    if status == 0:
        mess = "PLAY"
    elif status == -1:
        mess = "YOU DIED (YOURSELF)"
    elif status == -2:
        mess = "YOU DIED (LAVA)"
    title.set_text(f"Score = {snake.score}, {mess}")
    plt.draw()
    return (im,)


snake = FastSnake(Nrow=10, Ncol=10)

fig1, ax = plt.subplots()
ax.axis("off")
title = plt.title(f"Score = {snake.score}, PLAY ")
im = plt.imshow(snake.grid, interpolation="nearest", animated=True)
# anim = animation.FuncAnimation(fig, updatefig, frames=400, interval=1, blit=True)
plt.show()
widgets.Box([left_widget, right_widget, up_widget, down_widget, reset_widget])

In [ ]:
plt.close()

## Automatic playing

At first, you are asked to build an agent that will play automatically. It will have to make decisions according to the snake's sensor values. These decisions are to choose the best direction to take.

In [ ]:
def my_agent(sensors):
    neighbors = sensors[:8]
    dcol, drow = sensors[8:]
    # FRUIT IS NEAR ?
    for direction in np.arange(4):
        if neighbors[2 * direction] == 1:
            return direction
    # OK, NO, THEN GET CLOSER TO IT ?
    possible_neighbors = neighbors[::2]
    if 0.5 in possible_neighbors:
        prefered_directions = np.zeros(4)
        if dcol > 0:
            prefered_directions[:] += 2, 1, 0, 1
        else:
            prefered_directions[:] += 0, 1, 2, 1
        if drow > 0.0:
            prefered_directions[:] += 1, 0, 1, 2
        else:
            prefered_directions[:] += 1, 2, 1, 0
        p = prefered_directions * (possible_neighbors > 0)
        my_choice = np.random.choice(np.where(p == p.max())[0])
        return my_choice
    else:
        # OK LET'S SUICIDE
        return np.random.randint(4)


my_choice = my_agent(snake.sensors())
my_choice

## Automatic play with graphic output


In [ ]:
snake2 = FastSnake(Nrow=15, Ncol=15)


def updatefig(*args):
    sensors = snake2.sensors()
    my_choice = my_agent(sensors)
    snake2.play(my_choice)
    im2.set_array(snake2.grid)
    if snake2.status != 0:
        snake2.reset()
    return (im,)


fig2, ax2 = plt.subplots()
ax2.axis("off")
im2 = plt.imshow(snake2.grid, interpolation="nearest", animated=True)
anim = animation.FuncAnimation(fig2, updatefig, frames=40, interval=50, blit=True)
plt.show()

In [ ]:
anim.pause()
plt.close(fig2)

## Benchmark

In [ ]:
import tqdm

Ntests = 200
max_turns = 1000
snake3 = FastSnake(Nrow=10, Ncol=10)
scores = np.zeros(Ntests)
turns = np.zeros(Ntests)
for test in tqdm.trange(Ntests):
    snake3.reset()
    turn = 0
    while snake3.status == 0:
        sensors = snake3.sensors()
        my_choice = my_agent(sensors)
        snake3.play(my_choice)
        turn += 1
        if turn >= max_turns:
            break
    scores[test] = snake3.score
    turns[test] = turn

In [ ]:
data = pd.DataFrame({"score": scores, "turns": turns})
data.describe()

## Genetic Neural Network agent 

In [ ]:
Npop = 2000
Ngen = 10
Nin = 10
Nout = 4
keep_ratio = 0.1
keep_individuals = int(keep_ratio * Npop)
max_turns = 200


# all_bias = np.random.rand(Npop, Nout)
all_weights = np.random.rand(Npop, Nout, Nin)


def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


def genetic_algorithm(weigths, data):
    data.sort_values("score", ascending=False, inplace=True)
    keep_individuals = int(keep_ratio * Npop)
    kept_data = data.iloc[:keep_individuals]
    new_all_weights = np.zeros_like(all_weights)
    new_all_weights[:keep_individuals] = all_weights[kept_data.index]
    keep_range = np.arange(keep_individuals)
    for indiv in range(keep_individuals, Npop):
        parents = np.random.choice(keep_range, 2)
        pw = np.random.rand()
        w = new_all_weights[parents[0]] * pw + (1.0 - pw) + new_all_weights[parents[1]]

    weights = new_all_weights
    return weights


def neural_agent(weights, sensors):
    out = sigmoid((weights @ sensors))  # + weights[-1])
    return np.where(out == out.max())[0][0]

In [ ]:
snake4 = FastSnake(Nrow=10, Ncol=10)
scores = np.zeros(Npop)
turns = np.zeros(Npop)
for generation in range(Ngen):
    scores *= 0.0
    turns *= 0.0
    for test in tqdm.trange(Npop):
        snake4.reset()
        turn = 0
        weights = all_weights[test]
        while snake4.status == 0:
            sensors = snake4.sensors()
            my_choice = neural_agent(weights, sensors)
            snake4.play(my_choice)
            turn += 1
            if turn >= max_turns:
                break
        scores[test] = snake4.score
        turns[test] = turn
    data = pd.DataFrame({"score": scores, "turns": turns})
    data["perf"] = data.score * data.turns
    # all_weights = genetic_algorithm(all_weights, data)
    data.sort_values("perf", ascending=False, inplace=True)

    kept_data = data.iloc[:keep_individuals]
    new_all_weights = np.zeros_like(all_weights)
    new_all_weights[:keep_individuals] = all_weights[kept_data.index]
    keep_range = np.arange(keep_individuals)
    for indiv in range(keep_individuals, Npop):
        parents = np.random.choice(keep_range, 2)
        pw = np.random.rand()
        new_all_weights[indiv] = (
            new_all_weights[parents[0]] * pw + (1.0 - pw) + new_all_weights[parents[1]]
        )

    weights = new_all_weights

kept_data.head(20)

In [ ]:
data.head(10)

In [ ]:
snake2 = FastSnake(Nrow=15, Ncol=15)

weights = all_weights[0]  # BEST AGENT


def updatefig(*args):
    sensors = snake2.sensors()
    my_choice = neural_agent(weights, sensors)
    snake2.play(my_choice)
    im2.set_array(snake2.grid)
    if snake2.status != 0:
        snake2.reset()
    return (im,)


fig2, ax2 = plt.subplots()
ax2.axis("off")
im2 = plt.imshow(snake2.grid, interpolation="nearest", animated=True)
anim = animation.FuncAnimation(fig2, updatefig, frames=40, interval=50, blit=True)
plt.show()

In [ ]:
anim.pause()